In [ ]:
import warnings
warnings.filterwarnings("ignore")

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
from PIL import Image
from sklearn.model_selection import train_test_split
import predicate
from route_optimization_ny import route_optimisation
from predicate import use_predicate_logic
from importlib import reload
reload(predicate)

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
from tensorflow.keras.utils import plot_model
from tensorflow.keras.models import Model
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array, load_img


from tensorflow.keras.applications import MobileNetV2

In [ ]:

print("Note: Give location in format - Location, New York") # We can do more locations but very far apart distance might not load in the map
start_location=input("Start Location: ")
end_location=input("End Location: ")

route_optimisation(start_location,end_location)

In [ ]:
# Base path to the dataset folder
base_path = '/Users/riyanshibohra/Desktop/Datasets' # Replace with your path

# Lists to store image paths
list_other = []
list_safe = []
list_talking = []
list_text = []
list_turn = []

# Paths for each category
categories = {
    'other_activities': list_other,
    'safe_driving': list_safe,
    'talking_phone': list_talking,
    'texting_phone': list_text,
    'turning': list_turn
}

# Loop through each category directory and append image paths
for category, image_list in categories.items():
    category_path = os.path.join(base_path, category)
    for image in os.listdir(category_path):
        if image.endswith(('.png', '.jpg')):
            full_path = os.path.join(category_path, image)
            image_list.append(full_path)
            print(full_path)

In [ ]:
# Verify the number of images in each category
for category, image_list in categories.items():
    print(f"{category} has {len(image_list)} images.")

#### Data Visualization for each class

In [ ]:
# Function to display images for a given category
def display_images(images, title):
    plt.figure(figsize=(10, 10))  
    plt.suptitle(title)  

    # Display up to 4 images
    for i in range(min(4, len(images))):
        img = Image.open(images[i])
        plt.subplot(2, 2, i + 1)  # Arrange plots in 2x2 grid
        plt.imshow(img)
        plt.axis('off')

    plt.show()  # Show the plot

In [ ]:
# Display images from each category with custom titles
for category, images in categories.items():
    title = f"{category.replace('_', ' ').title()} - Example Images"
    display_images(images, title)

#### Data Splitting

In [ ]:
# Combine into a single list of image paths and labels
image_paths = list_other + list_safe + list_talking + list_text + list_turn
labels = [1] * len(list_other) + [0] * len(list_safe) + [4] * len(list_talking) + [3] * len(list_text) + [2] * len(list_turn)

In [ ]:

# First split to separate out the test set
X_train_val, X_test, y_train_val, y_test = train_test_split(
    image_paths, labels, test_size=0.15, random_state=42, stratify=labels)

# Second split to separate out the validation set from the remaining training set
X_train, X_val, y_train, y_val = train_test_split(
    X_train_val, y_train_val, test_size=0.1765, random_state=42, stratify=y_train_val)  # 0.1765 ≈ 15/85



In [ ]:
def check_distribution(labels, title):
    from collections import Counter
    distribution = Counter(labels)
    total = sum(distribution.values())
    for k in sorted(distribution):
        print(f"Class {k}: {distribution[k]/total:.2%}")
    print(f"Total samples in {title}: {total}")

check_distribution(y_train, "Training Set")
check_distribution(y_val, "Validation Set")
check_distribution(y_test, "Test Set")

In [ ]:
# Create dataframes from the splits
train_df = pd.DataFrame({'Image_Path': X_train, 'Label': y_train})
val_df = pd.DataFrame({'Image_Path': X_val, 'Label': y_val})
test_df = pd.DataFrame({'Image_Path': X_test, 'Label': y_test})

In [ ]:
def print_label_distribution(df, dataset_name):
    print(f"Label Distribution in {dataset_name}:")
    print(df['Label'].value_counts(normalize=True))  # This prints the percentage of each label

print_label_distribution(train_df, "Training Data")
print_label_distribution(val_df, "Validation Data")
print_label_distribution(test_df, "Test Data")

In [ ]:
def show_sample_images(df, num_images=4):
    sample = df.sample(n=num_images)
    plt.figure(figsize=(10, 10))
    for index, row in enumerate(sample.itertuples(), 1):
        img = Image.open(row.Image_Path)
        plt.subplot(2, 2, index)
        plt.imshow(img)
        plt.title(f"Class: {row.Label}")
        plt.axis('off')
    plt.show()

show_sample_images(train_df)

In [ ]:
# Shuffle DataFrames
train_df = train_df.sample(frac=1).reset_index(drop=True)
val_df = val_df.sample(frac=1).reset_index(drop=True)
test_df = test_df.sample(frac=1).reset_index(drop=True)

In [ ]:
# Defining Hyperparameters (Batch_Size, Height and Width)

Batch_size = 64
Img_height = 240
Img_width = 240

In [ ]:
# Rescaling the images to put all images in same shape for input layer of the model

train_datagen = ImageDataGenerator(rescale=1./255.)
test_datagen = ImageDataGenerator(rescale=1./255.)
val_datagen = ImageDataGenerator(rescale=1./255.)

In [ ]:
# Convert integer labels in the dataframe to strings

train_df['Label'] = train_df['Label'].astype(str)
val_df['Label'] = val_df['Label'].astype(str)
test_df['Label'] = test_df['Label'].astype(str)

In [ ]:
trainDataset = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    x_col='Image_Path',  
    y_col='Label',       
    target_size=(Img_height, Img_width),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=Batch_size,
    shuffle=True,
    seed=42
)

valDataset = test_datagen.flow_from_dataframe(
    dataframe=val_df,
    x_col='Image_Path',
    y_col='Label',
    target_size=(Img_height, Img_width),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=Batch_size,
    shuffle=True,
    seed=42
)

testDataset = val_datagen.flow_from_dataframe(
    dataframe=test_df,
    x_col='Image_Path',
    y_col='Label',
    target_size=(Img_height, Img_width),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=Batch_size,
    shuffle=False,  # Generally, shuffling is not required for testing.
    seed=42
)


### Model Development

#### MobileNetV2

In [ ]:
def create_transfer_learning_model(input_shape=(240, 240, 3), num_classes=5):
    base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=input_shape)
    base_model.trainable = False  # Freeze the base model
    
    model = models.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation='softmax')  # Output layer
    ])
    
    return model

# Create and compile the model
model = create_transfer_learning_model()
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

In [ ]:
history = model.fit(trainDataset,
                    epochs=20,  # You can adjust this based on the training progress
                    validation_data=valDataset)

#### Saving the Model

In [ ]:
# Assuming 'model' is your trained model (Create directory for your pc)
model.save('/Users/riyanshibohra/Desktop/AI Project/my_model.h5') 

In [ ]:
# Load the model
model = load_model('/Users/riyanshibohra/Desktop/AI Project/my_model.h5') #Replace with the actual path to your model

#### Making Predictions on Test Dataset

In [ ]:
# Assuming you have a test dataset called 'testDataset'
predictions = model.predict(testDataset)
predicted_classes = np.argmax(predictions, axis=1)

In [ ]:
# Assuming 'testDataset' is a tf.data.Dataset or a generator that yields (input, label) pairs
loss, accuracy = model.evaluate(testDataset)
print(f"Accuracy on the test set: {accuracy * 100:.2f}%")

In [ ]:

training_loss = history.history['loss']
val_loss = history.history['val_loss']
training_acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
epochs = range(1, len(training_loss) + 1)  # Range of epochs

# Set up the matplotlib figure and axes, specifying the size and DPI for better resolution
plt.figure(figsize=(10, 5), dpi=200)

# Plot training & validation loss values
plt.subplot(1, 2, 1)  # 1 row, 2 columns, first subplot
plt.plot(epochs, training_loss, 'navy', label='Training Loss')
plt.plot(epochs, val_loss, 'orangered', label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.xticks(np.arange(1, len(epochs)+1, 1))  # Ensure ticks correspond to epochs
plt.legend()

# Plot training & validation accuracy values
plt.subplot(1, 2, 2)  # 1 row, 2 columns, second subplot
plt.plot(epochs, training_acc, 'navy', label='Training Accuracy')
plt.plot(epochs, val_acc, 'orangered', label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.xticks(np.arange(1, len(epochs)+1, 1))  # Ensure ticks correspond to epochs
plt.legend()

# Display the plots
plt.tight_layout()
plt.show()

#### Testing the model on unseen images

In [ ]:
# Load the image
sample_image_path = '/Users/riyanshibohra/Desktop/AI Project/safe_driving.jpeg'  # Replace with the actual path to your image
image = load_img(sample_image_path, target_size=(240, 240))  # Adjust size if necessary
image = img_to_array(image)
image = np.expand_dims(image, axis=0)  # Model expects a batch

# Preprocess the image as you did for your training data
# Below is a simple rescaling example; adjust according to your preprocessing steps
image = image / 255.0

In [ ]:
prediction = model.predict(image)
predicted_class = np.argmax(prediction, axis=1)[0]  # Get the numerical label
print(f"The image is classified as class: {predicted_class}")

In [ ]:
# Load the image
sample_image_path2 = '/Users/riyanshibohra/Desktop/AI Project/test1.jpg'  # Replace with the actual path to your image
image2 = load_img(sample_image_path2, target_size=(240, 240))  # Adjust size if necessary
image2 = img_to_array(image2)
image2 = np.expand_dims(image2, axis=0)  # Model expects a batch

# Preprocess the image as you did for your training data
# Below is a simple rescaling example; adjust according to your preprocessing steps
image2 = image2 / 255.0

In [ ]:
prediction2 = model.predict(image2)
predicted_class2 = np.argmax(prediction2, axis=1)[0]  # Get the numerical label
print(f"The image is classified as class: {predicted_class2}")

In [ ]:
np.argmax(prediction2, axis=1)[0]

In [19]:
use_predicate_logic(prediction,start_location,end_location)

Attention Required - Face Forward
